In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
#import dask.dataframe as dd
#!pip install fastparquet
#from fastparquet import ParquetFile

import re
import nltk
import json
import pandas as pd
import numpy as np
!pip install vaderSentiment
from nltk.corpus import wordnet 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

nltk.download('wordnet')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)

#imports to get our parquet files aand dask files

     |████████████████████████████████| 133kB 2.7MB/s 
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Load pandas dataframe
def load_data(url):
    if url.endswith(".csv") or "csv" in url:
        df = pd.read_csv(url)
    else:
        df = None
    return df


In [ ]:
rev_url = 'https://tally-ai-dspt3.s3.amazonaws.com/yelp-restaurants/reviews-6-15-20.csv'

rev = load_data(rev_url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
rev.head()

,Unnamed: 0,review_id,business_id,reviews_stars,date,text
0,0,Oa-sapLub0wRAzxRKVmtDQ,j9bWpCRwpDVfwVT_V85qeA,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,1,NawvdCkVIvhMO_eMVMdBhw,j9bWpCRwpDVfwVT_V85qeA,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,2,uBWpaM4_PQ1NMB3N9nYkJA,j9bWpCRwpDVfwVT_V85qeA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,3,EIcE_Xw9G5bDAY-XQ0Vosg,j9bWpCRwpDVfwVT_V85qeA,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,4,BXTxF_ijBmm5RPYELr-5RQ,j9bWpCRwpDVfwVT_V85qeA,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [ ]:
rev.columns

Index(['Unnamed: 0', 'review_id', 'business_id', 'reviews_stars', 'date',
       'text'],
      dtype='object')

In [ ]:
rev = rev.drop(columns=["Unnamed: 0"])
rev.head()

,review_id,business_id,reviews_stars,date,text
0,Oa-sapLub0wRAzxRKVmtDQ,j9bWpCRwpDVfwVT_V85qeA,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,NawvdCkVIvhMO_eMVMdBhw,j9bWpCRwpDVfwVT_V85qeA,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,uBWpaM4_PQ1NMB3N9nYkJA,j9bWpCRwpDVfwVT_V85qeA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,EIcE_Xw9G5bDAY-XQ0Vosg,j9bWpCRwpDVfwVT_V85qeA,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,BXTxF_ijBmm5RPYELr-5RQ,j9bWpCRwpDVfwVT_V85qeA,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [ ]:
rev.dtypes

review_id         object
business_id       object
reviews_stars    float64
date              object
text              object
dtype: object

In [ ]:
#causes errors when running sentiment analysis
rev['text'] = rev['text'].astype(str)

In [ ]:
rev['text'] = rev['text'].str.replace('\d+', '')

In [ ]:
rev['text'] = rev['text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

In [ ]:
rev.shape
#3,244,807 reviews

(3244807, 5)

# Try NLP on segments of reviews df

In [ ]:
sent3 = rev.iloc[300001:350000]
sent3.head()

,review_id,business_id,reviews_stars,date,text
300001,QtdfgzU2QWqkqqnSQSJsCg,bjJcDCx5uMUr89nxBdGizw,5.0,2017-11-26 18:06:00,This place is legit The breakfast was amazing. The staff so attentive and on top of it. Thanks Brittany
300002,YA_UscHihJW0Z7YSvPt_9A,bjJcDCx5uMUr89nxBdGizw,5.0,2018-03-28 20:52:58,Awesome service and tasty food. Husband had the Hawaiian pork chops and egg breakfast. I had the waffle with eggs bacon and added strawberries bananas to the waffle. The waffle is so good it didnt need the fruit. Next time Im having it plain. Staff are genuinely nice and inviting. Definitely worth giving this place your business.
300003,10gJft08qwWrVXm5DgeA3w,bjJcDCx5uMUr89nxBdGizw,5.0,2018-03-22 17:01:43,WOW Just WOW This place knocks it outta the park.\nSOOOOO lucky to have this family owned joint here in Henderson-
300004,8QoGf7QsDVOjkRHOfewY1w,bjJcDCx5uMUr89nxBdGizw,5.0,2017-11-14 18:25:17,My husband and I came here for breakfast and seriously was the best experience Everyone was extremely nice and the food blew us away Came out fast and extremely delicious. Oh and the caramel bananas were bomb.com
300005,nvX0xUUgJhkoG9hrYJEyLQ,bjJcDCx5uMUr89nxBdGizw,5.0,2017-09-25 03:24:16,"Loved the first breakfast last week and have been back twice more to try additional breakfast entrees. The Belgian waffle was outstanding and the chili relleno and eggs were super delicious. The service, quality of food, and ambiance is very consistent. I am anxious to try the lunch menu soon.\n\nThe best breakfast restaurant I have been to since moving here years ago. The décor is super awesome, along with the friendly service from Gabriela I felt like I was back home in downtown Chicago. This place is clean with a friendly staff. Breakfast and Lunch served all day. Everything is fresh and delicious. Finally found a place that can make hash browns the right way. Biscuits and gravy were outstanding. I plan on going back often"


In [ ]:
sent3.shape

(49999, 5)

In [ ]:
#load VADER
analyzer = SentimentIntensityAnalyzer()
#Add VADER metrics to dataframe

In [ ]:
sent3['compound'] = [analyzer.polarity_scores(v)['compound'] for v in sent3['text']]
sent3['neg'] = [analyzer.polarity_scores(v)['neg'] for v in sent3['text']]
sent3['neu'] = [analyzer.polarity_scores(v)['neu'] for v in sent3['text']]
sent3['pos'] = [analyzer.polarity_scores(v)['pos'] for v in sent3['text']]

sent3.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,review_id,business_id,reviews_stars,date,text,compound,neg,neu,pos
300001,QtdfgzU2QWqkqqnSQSJsCg,bjJcDCx5uMUr89nxBdGizw,5.0,2017-11-26 18:06:00,This place is legit The breakfast was amazing. The staff so attentive and on top of it. Thanks Brittany,0.8176,0.0,0.653,0.347
300002,YA_UscHihJW0Z7YSvPt_9A,bjJcDCx5uMUr89nxBdGizw,5.0,2018-03-28 20:52:58,Awesome service and tasty food. Husband had the Hawaiian pork chops and egg breakfast. I had the waffle with eggs bacon and added strawberries bananas to the waffle. The waffle is so good it didnt need the fruit. Next time Im having it plain. Staff are genuinely nice and inviting. Definitely worth giving this place your business.,0.9580,0.0,0.715,0.285
300003,10gJft08qwWrVXm5DgeA3w,bjJcDCx5uMUr89nxBdGizw,5.0,2018-03-22 17:01:43,WOW Just WOW This place knocks it outta the park.\nSOOOOO lucky to have this family owned joint here in Henderson-,0.9164,0.0,0.603,0.397


In [ ]:
sent3['text_length'] = sent3['text'].str.len()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
columns = ['review_id', 'business_id', 'compound', 'text']

In [ ]:
df = sent3[columns]

In [ ]:
def tokenizer(doc):
     return [token for token in simple_preprocess(doc) 
             if token not in STOPWORDS]


def related_to_food(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('food.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_service(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('service.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_speed(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('speed.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list

def related_to_price(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('price.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_ambience(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('ambience.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_experience(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('experience.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list

def extract_subject_related_words():
  df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
  df['cleaned'] = df['text'].apply(tokenizer)
  df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
  df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
  df['words_related_to_speed'] = df['cleaned'].apply(related_to_speed)
  df['words_related_to_price'] = df['cleaned'].apply(related_to_price)
  df['words_related_to_ambience'] = df['cleaned'].apply(related_to_ambience)
  df['words_related_to_experience'] = df['cleaned'].apply(related_to_experience)

extract_subject_related_words()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
df.head()

,review_id,business_id,compound,text,cleaned,words_related_to_food,words_related_to_service,words_related_to_speed,words_related_to_price,words_related_to_ambience,words_related_to_experience
300001,QtdfgzU2QWqkqqnSQSJsCg,bjJcDCx5uMUr89nxBdGizw,0.8176,This place is legit The breakfast was amazing The staff so attentive and on top of it Thanks Brittany,"[place, legit, breakfast, amazing, staff, attentive, thanks, brittany]",[breakfast],[],[],[],[],[]
300002,YA_UscHihJW0Z7YSvPt_9A,bjJcDCx5uMUr89nxBdGizw,0.9580,Awesome service and tasty food Husband had the Hawaiian pork chops and egg breakfast I had the waffle with eggs bacon and added strawberries bananas to the waffle The waffle is so good it didnt need the fruit Next time Im having it plain Staff are genuinely nice and inviting Definitely worth giving this place your business,"[awesome, service, tasty, food, husband, hawaiian, pork, chops, egg, breakfast, waffle, eggs, bacon, added, strawberries, bananas, waffle, waffle, good, didnt, need, fruit, time, im, having, plain, staff, genuinely, nice, inviting, definitely, worth, giving, place, business]","[food, breakfast, eggs]","[service, time, giving]",[],[good],[need],[]
300003,10gJft08qwWrVXm5DgeA3w,bjJcDCx5uMUr89nxBdGizw,0.9164,WOW Just WOW This place knocks it outta the park SOOOOO lucky to have this family owned joint here in Henderson,"[wow, wow, place, knocks, outta, park, sooooo, lucky, family, owned, joint, henderson]",[],[knocks],[],[],[],[]
300004,8QoGf7QsDVOjkRHOfewY1w,bjJcDCx5uMUr89nxBdGizw,0.8906,My husband and I came here for breakfast and seriously was the best experience Everyone was extremely nice and the food blew us away Came out fast and extremely delicious Oh and the caramel bananas were bomb com,"[husband, came, breakfast, seriously, best, experience, extremely, nice, food, blew, away, came, fast, extremely, delicious, oh, caramel, bananas, bomb, com]","[breakfast, food, caramel]","[best, fast]",[],[],[],[experience]
300005,nvX0xUUgJhkoG9hrYJEyLQ,bjJcDCx5uMUr89nxBdGizw,0.9937,Loved the first breakfast last week and have been back twice more to try additional breakfast entrees The Belgian waffle was outstanding and the chili relleno and eggs were super delicious The service quality of food and ambiance is very consistent I am anxious to try the lunch menu soon The best breakfast restaurant I have been to since moving here years ago The d cor is super awesome along with the friendly service from Gabriela I felt like I was back home in downtown Chicago This place is clean with a friendly staff Breakfast and Lunch served all day Everything is fresh and delicious Finally found a place that can make hash browns the right way Biscuits and gravy were outstanding I plan on going back often,"[loved, breakfast, week, twice, try, additional, breakfast, entrees, belgian, waffle, outstanding, chili, relleno, eggs, super, delicious, service, quality, food, ambiance, consistent, anxious, try, lunch, menu, soon, best, breakfast, restaurant, moving, years, ago, cor, super, awesome, friendly, service, gabriela, felt, like, home, downtown, chicago, place, clean, friendly, staff, breakfast, lunch, served, day, fresh, delicious, finally, place, hash, browns, right, way, biscuits, gravy, outstanding, plan, going]","[breakfast, breakfast, entrees, chili, eggs, food, lunch, breakfast, breakfast, lunch, hash, biscuits, gravy]","[try, service, try, best, service, clean, going]",[],[quality],"[quality, ambiance, way]","[like, right, plan]"


In [ ]:
def sentiment_score(sentence):
  # Create a SentimentIntensityAnalyzer object. 
  sid_obj = SentimentIntensityAnalyzer()
  # polarity_scores method of SentimentIntensityAnalyzer oject gives a sentiment dictionary. which contains pos, neg, neu, and compound scores. 
  sentiment_dict = sid_obj.polarity_scores(sentence)
  return sentiment_dict

def get_sentiment(review_list):
  all_sentiments = []
  compounds = []
  if len(review_list) > 0:
    for review in review_list:
      score = sentiment_score(review)
      all_sentiments.append(score)

  if len(all_sentiments) > 0:
    for sentiment_dict in all_sentiments:
      compound = sentiment_dict['compound']
      compounds.append(compound)

  if len(compounds) > 0:
    avg_sentiment = sum(compounds) / len(compounds)
	
  else:
    avg_sentiment = None
    
  return avg_sentiment

def get_subject_review_scores(review_list):
  if len(review_list) > 0:
    sub_sentiment_score = round((get_sentiment(review_list))*150)
    if sub_sentiment_score < 0:
      sub_sentiment_score = 0
    else:
      sub_sentiment_score = 75 #leaving at 75 to give neutral score
    return sub_sentiment_score

def sentimental_analysis(df):
  # Create a SentimentIntensityAnalyzer object. 
  sid_obj = SentimentIntensityAnalyzer()
  # extract_subject_related_words
  df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
  df['cleaned'] = df['text'].apply(tokenizer)

  df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
  df['food_review_list'] = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
  df['food_score'] = df['food_review_list'].apply(get_subject_review_scores)

  df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
  df['service_review_list'] = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
  df['service_score'] = df['service_review_list'].apply(get_subject_review_scores)

  df['words_related_to_speed'] = df['cleaned'].apply(related_to_service)
  df['speed_review_list'] = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
  df['speed_score'] = df['speed_review_list'].apply(get_subject_review_scores)

  df['words_related_to_price'] = df['cleaned'].apply(related_to_service)
  df['price_review_list'] = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
  df['price_score'] = df['price_review_list'].apply(get_subject_review_scores)

  df['words_related_to_ambience'] = df['cleaned'].apply(related_to_service)
  df['ambience_review_list'] = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
  df['ambience_score'] = df['ambience_review_list'].apply(get_subject_review_scores)

  df['words_related_to_experience'] = df['cleaned'].apply(related_to_service)
  df['experience_review_list'] = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()
  df['experience_score'] = df['experience_review_list'].apply(get_subject_review_scores)
  
  return df


In [ ]:
df.head()

,review_id,business_id,compound,text,cleaned,words_related_to_food,words_related_to_service,words_related_to_speed,words_related_to_price,words_related_to_ambience,words_related_to_experience
300001,QtdfgzU2QWqkqqnSQSJsCg,bjJcDCx5uMUr89nxBdGizw,0.8176,This place is legit The breakfast was amazing The staff so attentive and on top of it Thanks Brittany,"[place, legit, breakfast, amazing, staff, attentive, thanks, brittany]",[breakfast],[],[],[],[],[]
300002,YA_UscHihJW0Z7YSvPt_9A,bjJcDCx5uMUr89nxBdGizw,0.9580,Awesome service and tasty food Husband had the Hawaiian pork chops and egg breakfast I had the waffle with eggs bacon and added strawberries bananas to the waffle The waffle is so good it didnt need the fruit Next time Im having it plain Staff are genuinely nice and inviting Definitely worth giving this place your business,"[awesome, service, tasty, food, husband, hawaiian, pork, chops, egg, breakfast, waffle, eggs, bacon, added, strawberries, bananas, waffle, waffle, good, didnt, need, fruit, time, im, having, plain, staff, genuinely, nice, inviting, definitely, worth, giving, place, business]","[food, breakfast, eggs]","[service, time, giving]",[],[good],[need],[]
300003,10gJft08qwWrVXm5DgeA3w,bjJcDCx5uMUr89nxBdGizw,0.9164,WOW Just WOW This place knocks it outta the park SOOOOO lucky to have this family owned joint here in Henderson,"[wow, wow, place, knocks, outta, park, sooooo, lucky, family, owned, joint, henderson]",[],[knocks],[],[],[],[]
300004,8QoGf7QsDVOjkRHOfewY1w,bjJcDCx5uMUr89nxBdGizw,0.8906,My husband and I came here for breakfast and seriously was the best experience Everyone was extremely nice and the food blew us away Came out fast and extremely delicious Oh and the caramel bananas were bomb com,"[husband, came, breakfast, seriously, best, experience, extremely, nice, food, blew, away, came, fast, extremely, delicious, oh, caramel, bananas, bomb, com]","[breakfast, food, caramel]","[best, fast]",[],[],[],[experience]
300005,nvX0xUUgJhkoG9hrYJEyLQ,bjJcDCx5uMUr89nxBdGizw,0.9937,Loved the first breakfast last week and have been back twice more to try additional breakfast entrees The Belgian waffle was outstanding and the chili relleno and eggs were super delicious The service quality of food and ambiance is very consistent I am anxious to try the lunch menu soon The best breakfast restaurant I have been to since moving here years ago The d cor is super awesome along with the friendly service from Gabriela I felt like I was back home in downtown Chicago This place is clean with a friendly staff Breakfast and Lunch served all day Everything is fresh and delicious Finally found a place that can make hash browns the right way Biscuits and gravy were outstanding I plan on going back often,"[loved, breakfast, week, twice, try, additional, breakfast, entrees, belgian, waffle, outstanding, chili, relleno, eggs, super, delicious, service, quality, food, ambiance, consistent, anxious, try, lunch, menu, soon, best, breakfast, restaurant, moving, years, ago, cor, super, awesome, friendly, service, gabriela, felt, like, home, downtown, chicago, place, clean, friendly, staff, breakfast, lunch, served, day, fresh, delicious, finally, place, hash, browns, right, way, biscuits, gravy, outstanding, plan, going]","[breakfast, breakfast, entrees, chili, eggs, food, lunch, breakfast, breakfast, lunch, hash, biscuits, gravy]","[try, service, try, best, service, clean, going]",[],[quality],"[quality, ambiance, way]","[like, right, plan]"


In [ ]:
df = df.reset_index()

In [ ]:
df.head()

,index,review_id,business_id,compound,text,cleaned,words_related_to_food,words_related_to_service,words_related_to_speed,words_related_to_price,words_related_to_ambience,words_related_to_experience
0,300001,QtdfgzU2QWqkqqnSQSJsCg,bjJcDCx5uMUr89nxBdGizw,0.8176,This place is legit The breakfast was amazing The staff so attentive and on top of it Thanks Brittany,"[place, legit, breakfast, amazing, staff, attentive, thanks, brittany]",[breakfast],[],[],[],[],[]
1,300002,YA_UscHihJW0Z7YSvPt_9A,bjJcDCx5uMUr89nxBdGizw,0.9580,Awesome service and tasty food Husband had the Hawaiian pork chops and egg breakfast I had the waffle with eggs bacon and added strawberries bananas to the waffle The waffle is so good it didnt need the fruit Next time Im having it plain Staff are genuinely nice and inviting Definitely worth giving this place your business,"[awesome, service, tasty, food, husband, hawaiian, pork, chops, egg, breakfast, waffle, eggs, bacon, added, strawberries, bananas, waffle, waffle, good, didnt, need, fruit, time, im, having, plain, staff, genuinely, nice, inviting, definitely, worth, giving, place, business]","[food, breakfast, eggs]","[service, time, giving]",[],[good],[need],[]
2,300003,10gJft08qwWrVXm5DgeA3w,bjJcDCx5uMUr89nxBdGizw,0.9164,WOW Just WOW This place knocks it outta the park SOOOOO lucky to have this family owned joint here in Henderson,"[wow, wow, place, knocks, outta, park, sooooo, lucky, family, owned, joint, henderson]",[],[knocks],[],[],[],[]
3,300004,8QoGf7QsDVOjkRHOfewY1w,bjJcDCx5uMUr89nxBdGizw,0.8906,My husband and I came here for breakfast and seriously was the best experience Everyone was extremely nice and the food blew us away Came out fast and extremely delicious Oh and the caramel bananas were bomb com,"[husband, came, breakfast, seriously, best, experience, extremely, nice, food, blew, away, came, fast, extremely, delicious, oh, caramel, bananas, bomb, com]","[breakfast, food, caramel]","[best, fast]",[],[],[],[experience]
4,300005,nvX0xUUgJhkoG9hrYJEyLQ,bjJcDCx5uMUr89nxBdGizw,0.9937,Loved the first breakfast last week and have been back twice more to try additional breakfast entrees The Belgian waffle was outstanding and the chili relleno and eggs were super delicious The service quality of food and ambiance is very consistent I am anxious to try the lunch menu soon The best breakfast restaurant I have been to since moving here years ago The d cor is super awesome along with the friendly service from Gabriela I felt like I was back home in downtown Chicago This place is clean with a friendly staff Breakfast and Lunch served all day Everything is fresh and delicious Finally found a place that can make hash browns the right way Biscuits and gravy were outstanding I plan on going back often,"[loved, breakfast, week, twice, try, additional, breakfast, entrees, belgian, waffle, outstanding, chili, relleno, eggs, super, delicious, service, quality, food, ambiance, consistent, anxious, try, lunch, menu, soon, best, breakfast, restaurant, moving, years, ago, cor, super, awesome, friendly, service, gabriela, felt, like, home, downtown, chicago, place, clean, friendly, staff, breakfast, lunch, served, day, fresh, delicious, finally, place, hash, browns, right, way, biscuits, gravy, outstanding, plan, going]","[breakfast, breakfast, entrees, chili, eggs, food, lunch, breakfast, breakfast, lunch, hash, biscuits, gravy]","[try, service, try, best, service, clean, going]",[],[quality],"[quality, ambiance, way]","[like, right, plan]"


In [ ]:
df = df.drop(columns=["index"])

In [ ]:
df.head()

,review_id,business_id,compound,text,cleaned,words_related_to_food,words_related_to_service,words_related_to_speed,words_related_to_price,words_related_to_ambience,words_related_to_experience
0,QtdfgzU2QWqkqqnSQSJsCg,bjJcDCx5uMUr89nxBdGizw,0.8176,This place is legit The breakfast was amazing The staff so attentive and on top of it Thanks Brittany,"[place, legit, breakfast, amazing, staff, attentive, thanks, brittany]",[breakfast],[],[],[],[],[]
1,YA_UscHihJW0Z7YSvPt_9A,bjJcDCx5uMUr89nxBdGizw,0.9580,Awesome service and tasty food Husband had the Hawaiian pork chops and egg breakfast I had the waffle with eggs bacon and added strawberries bananas to the waffle The waffle is so good it didnt need the fruit Next time Im having it plain Staff are genuinely nice and inviting Definitely worth giving this place your business,"[awesome, service, tasty, food, husband, hawaiian, pork, chops, egg, breakfast, waffle, eggs, bacon, added, strawberries, bananas, waffle, waffle, good, didnt, need, fruit, time, im, having, plain, staff, genuinely, nice, inviting, definitely, worth, giving, place, business]","[food, breakfast, eggs]","[service, time, giving]",[],[good],[need],[]
2,10gJft08qwWrVXm5DgeA3w,bjJcDCx5uMUr89nxBdGizw,0.9164,WOW Just WOW This place knocks it outta the park SOOOOO lucky to have this family owned joint here in Henderson,"[wow, wow, place, knocks, outta, park, sooooo, lucky, family, owned, joint, henderson]",[],[knocks],[],[],[],[]
3,8QoGf7QsDVOjkRHOfewY1w,bjJcDCx5uMUr89nxBdGizw,0.8906,My husband and I came here for breakfast and seriously was the best experience Everyone was extremely nice and the food blew us away Came out fast and extremely delicious Oh and the caramel bananas were bomb com,"[husband, came, breakfast, seriously, best, experience, extremely, nice, food, blew, away, came, fast, extremely, delicious, oh, caramel, bananas, bomb, com]","[breakfast, food, caramel]","[best, fast]",[],[],[],[experience]
4,nvX0xUUgJhkoG9hrYJEyLQ,bjJcDCx5uMUr89nxBdGizw,0.9937,Loved the first breakfast last week and have been back twice more to try additional breakfast entrees The Belgian waffle was outstanding and the chili relleno and eggs were super delicious The service quality of food and ambiance is very consistent I am anxious to try the lunch menu soon The best breakfast restaurant I have been to since moving here years ago The d cor is super awesome along with the friendly service from Gabriela I felt like I was back home in downtown Chicago This place is clean with a friendly staff Breakfast and Lunch served all day Everything is fresh and delicious Finally found a place that can make hash browns the right way Biscuits and gravy were outstanding I plan on going back often,"[loved, breakfast, week, twice, try, additional, breakfast, entrees, belgian, waffle, outstanding, chili, relleno, eggs, super, delicious, service, quality, food, ambiance, consistent, anxious, try, lunch, menu, soon, best, breakfast, restaurant, moving, years, ago, cor, super, awesome, friendly, service, gabriela, felt, like, home, downtown, chicago, place, clean, friendly, staff, breakfast, lunch, served, day, fresh, delicious, finally, place, hash, browns, right, way, biscuits, gravy, outstanding, plan, going]","[breakfast, breakfast, entrees, chili, eggs, food, lunch, breakfast, breakfast, lunch, hash, biscuits, gravy]","[try, service, try, best, service, clean, going]",[],[quality],"[quality, ambiance, way]","[like, right, plan]"


In [ ]:
df = sentimental_analysis(df)

ValueError: ignored

In [ ]:
#errored out when executed
df = df.apply(sentimental_analysis(df))

In [ ]:
df.to_csv('sentiment300001-350000.csv')
!cp sentiment300001-350000.csv "drive/My Drive/"